In [1]:
import numpy as np
import torch
import pickle
from torch_geometric.data import Data
import random

In [2]:
# Parameters
HIDDEN_DIM = 256
file_name = 'syn1'
use_ratio = 1

In [3]:
with open(f'../MLdebugger/data/{file_name}_journeys.pkl', 'rb') as f:
    journeys = pickle.load(f)
    journeys = random.sample(journeys, int(len(journeys) * use_ratio))

with open(f'../MLdebugger/data/{file_name}_combined_graph.pkl', 'rb') as f:
    graph = pickle.load(f)

In [4]:
# one hot encoded embedding strategy
def OHembed(num, dim):
    vec = np.zeros(dim)
    vec[num] = 1
    return vec

In [5]:
from torch.nn.utils.rnn import pad_sequence

In [6]:
# check the node types
types = set()
for i in graph.nodes:
    types.add(graph.nodes[i]['node_type'])

# create type and label -> id mapping
type2id = {}
for t in types:
    type2id[t] = len(type2id)

label2id = {}
for i, node in enumerate(graph):
    if graph.nodes[node]['label'] not in label2id:
        label2id[graph.nodes[node]['label']] = len(label2id)

# create node idx to id mapping
node2id = {}
for i, node in enumerate(graph):
    node2id[node] = i

# create edge list
edge_list_dep = []
edge_list_rev = []

for node in graph.nodes:
    for neighbor in graph.neighbors(node):
        edge_list_dep.append(node2id[node])
        edge_list_rev.append(node2id[neighbor])
        
# create node feature list
featureMatrix = []

for node in graph.nodes:
    #featureMatrix.append([type2id[graph.nodes[node]['node_type']], label2id[graph.nodes[node]['label']]])
    #featureMatrix.append(embed(label2id[graph.nodes[node]['label']], len(label2id)))
    featureMatrix.append(OHembed(label2id[graph.nodes[node]['label']], len(label2id)))

num_features = len(featureMatrix[0])

# jounrey to node id
journeys_id = []

for journey in journeys:
    journey_id = []
    for node in journey:
        journey_id.append(node2id[node])
    journeys_id.append(torch.tensor(journey_id, dtype=torch.long))

# trace in journey to fm
journeys_fm = []

for journey in journeys_id:
    journey_fm = []
    for node in journey:
        journey_fm.append(featureMatrix[node])
    journeys_fm.append(torch.tensor(journey_fm, dtype=torch.float))

# Pad the sequences so they are the same length
traces_x = pad_sequence(journeys_fm, batch_first=True)
traces_y = pad_sequence(journeys_id, batch_first=True)

# to tensor
featureMatrix = torch.tensor(featureMatrix, dtype=torch.float)
edge_list = torch.tensor([edge_list_dep, edge_list_rev], dtype=torch.long)

C:\Users\mnihy\AppData\Local\Temp\ipykernel_19244\1286467137.py:56: UserWarning: Creating a tensor from a list of numpy.ndarrays is extremely slow. Please consider converting the list to a single numpy.ndarray with numpy.array() before converting to a tensor. (Triggered internally at ..\torch\csrc\utils\tensor_new.cpp:248.)
  journeys_fm.append(torch.tensor(journey_fm, dtype=torch.float))


In [14]:
import torch.nn as nn
import torch.nn.functional as F
from torch_geometric.nn import SAGEConv

class BiLSTM(torch.nn.Module):
    def __init__(self, in_channels, hidden_channels, num_layers=1):
        super(BiLSTM, self).__init__()
        self.lstm = nn.LSTM(input_size=in_channels,
                            hidden_size=hidden_channels,
                            num_layers=num_layers,
                            bidirectional=True,
                            batch_first=True)
        
    def forward(self, x):
        x = x.unsqueeze(0)
        output, _ = self.lstm(x)
        # return output.squeeze(0)
        return output
    
class GraphSAGE(torch.nn.Module):
    def __init__(self, in_channels, hidden_channels):
        super(GraphSAGE, self).__init__()
        self.conv1 = SAGEConv(in_channels, hidden_channels)

    def forward(self, x, edge_index):
        x = self.conv1(x, edge_index)
        x = F.relu(x)
        return x

class CombinedModel(torch.nn.Module):
    def __init__(self, in_channels, hidden_channels):
        super(CombinedModel, self).__init__()
        self.graphsage = GraphSAGE(in_channels, hidden_channels)
        self.bilstm = BiLSTM(in_channels, hidden_channels//2)

    def forward(self, trace, x, edge_index):
        embeddings = self.graphsage(x, edge_index)
        out_bilstm = self.bilstm(trace)
        embeddings_expanded = embeddings.unsqueeze(0).expand(out_bilstm.size(0), -1, -1) # not original
        #combined = torch.matmul(embeddings, out_bilstm.transpose(0, 1))
        combined = torch.bmm(out_bilstm, embeddings_expanded.transpose(1, 2))
        return combined

device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
model = CombinedModel(num_features, HIDDEN_DIM).to(device)
featureMatrix = featureMatrix.to(device)
edge_list = edge_list.to(device)
traces_x = traces_x.to(device)
traces_y = traces_y.to(device)
optimizer = torch.optim.Adam(model.parameters(), lr=0.01, weight_decay=5e-4)

def train(trace_x, trace_y):
    model.train()
    optimizer.zero_grad()
    #print(trace_x.shape, trace_y.shape, featureMatrix.shape, edge_list.shape)
    combined = model(trace_x, featureMatrix, edge_list)
    # out = combined.transpose(0, 1)
    out = combined.view(-1, combined.size(-1))
    trace_y = trace_y.view(-1)
    #print(trace_x, trace_y)
    loss = F.cross_entropy(out, trace_y)
    loss.backward()
    optimizer.step()
    return loss.item()

def test(traces_x, traces_y):
    model.eval()
    # loop through each trace

    num_correct = 0
    num_questions = 0

    with torch.no_grad():
        for i in range(len(traces_x)):
            trace_x = traces_x[i]
            trace_y = traces_y[i]
            combined = model(trace_x, featureMatrix, edge_list)
            #out = combined.transpose(0, 1)
            #pred = out.argmax(dim=1)
            pred = combined.argmax(dim=2)
            num_correct += pred.eq(trace_y).sum()
            #num_questions += len(trace_y)
            num_questions += len(trace_y.view(-1))
    
    return num_correct / num_questions

for epoch in range(1, 201):
    for i in range(len(traces_x)):
        trace_x = traces_x[i]
        trace_y = traces_y[i]
        loss = train(trace_x, trace_y)
    if epoch % 10 == 0:
        acc = test(traces_x=traces_x, traces_y=traces_y)
        print(f'Epoch: {epoch}, Loss: {loss:.4f}, Test Acc: {acc:.4f}')


Epoch: 10, Loss: 0.0164, Test Acc: 1.0000


KeyboardInterrupt: 